In [1]:
import torch
import os
import wandb
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score


# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

# set repeatability
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)     # Set a random seed for CUDA operations.
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    
    # Ensure deterministic behavior for CUDA operations (note: If you are not concerned with reportable reproducibility, set deterministic to false, and benchmark to true - as it can choose faster algorithms).
    torch.backends.cudnn.deterministic = True  # Set cuDNN to deterministic mode - it will now only select algorithms that are known to be deterministic.
    torch.backends.cudnn.benchmark = False  # Disable cuDNN benchmarking - it may select the best algorithms for the hardware, but it doesn't guarantee deterministic results.   

Using device: cuda


In [2]:
# Load the data with column headers
column_names = ['id', 'longitude', 'latitude', 'value']
data = pd.read_csv('3D_spatial_network.txt', header=None, names=column_names)

# If you need to save as a proper CSV
data.to_csv('3D_spatial_network.csv', index=False)

# For RNN preparation
# Assuming you want to predict 'value' based on sequence of coordinates
# You'll need to create sequences from your data

In [ ]:
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = data[i+sequence_length][3]  # Assuming value is target
        sequences.append(seq[:, 1:3])  # Using just coordinates as features
        targets.append(label)
    return np.array(sequences), np.array(targets)

# Normalize the data first
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['longitude', 'latitude', 'value']])

# Create sequences
sequence_length = 10  # Adjust based on your needs
X, y = create_sequences(scaled_data, sequence_length)

# Split into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# For RNN input shape: [samples, timesteps, features]
# X_train shape will be (n_samples, sequence_length, 2) if using just coordinates

In [ ]:
# Load the API key from the environment variable
wandb.init(project="Assignment6", entity="usf-magma", config={
    "learning_rate": 0.00085,
    "dropout_percentage": 0.45,
    "batch_size": 64,
    "epochs": 15,
    "momentum": (0.9, 0.92),
    "weight_decay": 1e-5,
    "optimizer": "Adam",
    "criterion": "MSELoss",  # Changed to MSELoss for regression
    "input_size": 1,
    "hidden_size": 4
})

config = wandb.config

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=4, dropout=0.45):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, 
                         hidden_size=hidden_size, 
                         batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        rnn_out, hidden = self.rnn(x)
        last_time_step = rnn_out[:, -1, :]
        last_time_step = self.dropout(last_time_step)
        prediction = self.fc(last_time_step)
        return prediction

# Initialize wandb
wandb_api_key = os.getenv("WANDB_API_KEY")
if wandb_api_key:
    wandb.login(key=wandb_api_key)
else:
    raise ValueError("WANDB_API_KEY environment variable not set!")

# Initialize model with wandb config
model = SimpleRNN(
    input_size=config.input_size,
    hidden_size=config.hidden_size,
    dropout=config.dropout_percentage
)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.learning_rate,
    betas=config.momentum,
    weight_decay=config.weight_decay
)

# Example training loop
def train(model, dataloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(dataloader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            # Log batch-level metrics
            wandb.log({
                "batch_loss": loss.item(),
                "epoch": epoch,
                "batch": batch_idx
            })
        
        # Log epoch-level metrics
        avg_loss = total_loss / len(dataloader)
        wandb.log({
            "epoch_loss": avg_loss,
            "epoch": epoch
        })

# Example usage (you'll need to replace with your actual data loading)
if __name__ == "__main__":
    # Create sample data loader (replace with your actual data)
    # Normally you would create proper DataLoader with your sequences
    input_data = torch.tensor([[[32.0], [31.0], [30.0]]])
    target_data = torch.tensor([[29.5]])  # Example target
    
    # Create dummy dataset and dataloader
    dataset = torch.utils.data.TensorDataset(input_data, target_data)
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=config.batch_size,
        shuffle=True
    )
    
    # Train the model
    train(model, dataloader, criterion, optimizer, config.epochs)
    
    # Save model and finish wandb
    torch.save(model.state_dict(), "rnn_model.pth")
    wandb.save("rnn_model.pth")
    wandb.finish()